In [18]:
import numpy as np

class Node:
    def __init__(self, feature=None, value=None, result=None):
        self.feature = feature  # Index of feature to split on
        self.value = value      # Value of the feature to split on
        self.result = result    # Class label for leaf nodes
        self.children = {}      # Dictionary to store child nodes

def entropy(labels):
    unique_labels, counts = np.unique(labels, return_counts=True)
    probabilities = counts / len(labels)
    entropy_value = -np.sum(probabilities * np.log2(probabilities))
    return entropy_value

def information_gain(data, feature_index, threshold, labels):
    # Split the data based on the given feature and threshold
    left_mask = data[:, feature_index] == threshold
    right_mask = ~left_mask

    # Calculate entropy for the original data
    original_entropy = entropy(labels)

    # Calculate entropy for the left and right subsets
    left_entropy = entropy(labels[left_mask])
    right_entropy = entropy(labels[right_mask])

    # Calculate information gain
    gain = original_entropy - (len(labels[left_mask]) / len(labels) * left_entropy
                               + len(labels[right_mask]) / len(labels) * right_entropy)

    return gain

def find_best_split(data, labels):
    num_features = data.shape[1]
    best_feature = None
    best_threshold = None
    max_gain = -1

    for feature_index in range(num_features):
        unique_values = np.unique(data[:, feature_index])
        thresholds = unique_values

        for threshold in thresholds:
            gain = information_gain(data, feature_index, threshold, labels)

            if gain > max_gain:
                max_gain = gain
                best_feature = feature_index
                best_threshold = threshold

    return best_feature, best_threshold

def id3(data, labels, feature_names):
    # Base case: if all labels are the same, return a leaf node
    if len(np.unique(labels)) == 1:
        return Node(result=labels[0])

    # Base case: if no features left to split on, return a leaf node with the majority class
    if data.shape[1] == 0:
        majority_class = np.bincount(labels).argmax()
        return Node(result=majority_class)

    # Find the best feature and threshold to split on
    best_feature, best_threshold = find_best_split(data, labels)

    # Base case: if no useful split is found, return a leaf node with the majority class
    if best_feature is None:
        majority_class = np.bincount(labels).argmax()
        return Node(result=majority_class)

    # Split the data based on the best feature and threshold
    left_mask = data[:, best_feature] == best_threshold
    right_mask = ~left_mask

    # Recursively build the tree for the left and right subsets
    left_child = id3(data[left_mask], labels[left_mask], feature_names)
    right_child = id3(data[right_mask], labels[right_mask], feature_names)

    # Create a node for the best split
    node = Node(feature=feature_names[best_feature], value=best_threshold)
    node.children["{}".format(best_threshold)] = left_child
    node.children["not {}".format(best_threshold)] = right_child

    return node

def predict(node, sample, feature_names):
    if node.result is not None:
        return node.result

    if sample[feature_names.index(node.feature)] == node.value:
        return predict(node.children["{}".format(node.value)], sample, feature_names)
    else:
        return predict(node.children["not {}".format(node.value)], sample, feature_names)



In [25]:
# Example usage:
x = [['Sunny', 'Warm', 'High', 'Strong'],
     ['Sunny', 'Warm', 'Low', 'Weak'],
     ['Cloudy', 'Cool', 'High', 'Strong'],
     ['Cloudy', 'Cool', 'Low', 'Weak']]
y = ['No', 'Yes', 'No', 'Yes']
feature_names = ['Outlook', 'Temperature', 'Humidity', 'Wind']

# Convert categorical features to numerical values
for i in range(len(x[0])):
    unique_values = np.unique([row[i] for row in x])
    value_mapping = {value: idx for idx, value in enumerate(unique_values)}
    for j in range(len(x)):
        x[j][i] = value_mapping[x[j][i]]
print ("X",x)
# Convert labels to numerical values (0 and 1)
label_mapping = {'No': 0, 'Yes': 1}
y = [label_mapping[label] for label in y]
print("Y",y)

X [[2, 1, 0, 0], [2, 1, 1, 1], [0, 0, 0, 0], [1, 0, 1, 1]]
Y [0, 1, 0, 1]


In [23]:
data = np.array(x, dtype=int)
labels = np.array(y)

# Build the decision tree
tree = id3(data, labels, feature_names)

# Make predictions
sample = [0, 1, 0, 1]  # ['Sunny', 'Warm', 'Low', 'Weak']
prediction = predict(tree, sample, feature_names)
print("Prediction:", prediction)


Prediction: 0
